## 4장 서울시 셀프 주유소 가격분석
### Selenium 사용법

In [15]:
from selenium import webdriver # 브라우저를 제어하는 모듈

In [2]:
driver = webdriver.Chrome('./driver/chromedriver.exe')

**이제부터 코드를 통해서 제어**

In [3]:
# driver를 통해서 제어
driver.get('https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com')

제어를 하는 경우, 반드시 selenium을 써야 함!

### 아이디 입력 요소 선택

In [4]:
elem_login_id = driver.find_element_by_id('id') # 아이디가 id
elem_login_id.send_keys('spexle') # 해당 아이디 입력

In [5]:
elem_login_id.clear() # 입력된 id 제거

In [6]:
# 비밀번호 입력
elem_login_pw = driver.find_element_by_id('pw') # pw 선택
elem_login_pw.send_keys('123456789')

In [7]:
elem_login_pw.clear() # 입력된 pw 제거

### 로그인 버튼 클릭

In [8]:
#  로그인 버튼의 XPath : //*[@id="log.login"]
xPath = '''//*[@id="log.login"]''' # 작은 따옴표 3개여야 영향 nope!

login_btn = driver.find_element_by_xpath(xPath) # 로그인 버튼 제어 생성
login_btn.click()

### 브라우저 닫기 ( => 메모리 삭제)

In [9]:
# 메모리에 정상적으로 닫기 위해선 하나하나 닫아야 함! 
driver.close()

### 😨selenium으로 열린 브라우저는 절대 건들지 마세요!

# 서울시 구별 주유소 가격 정보 얻기!

### 드라이버 메모리 로딩

In [54]:
driver = webdriver.Chrome('./driver/chromedriver.exe')

In [57]:
driver.get('https://www.opinet.co.kr/searRgSelect.do')

In [58]:
# 시군구(Xpath)  =>  //*[@id="SIGUNGU_NM0"]
xpath = '''//*[@id="SIGUNGU_NM0"]'''
gu_list_raw = driver.find_element_by_xpath(xpath)

#### select box를 빼서 데이터를 추출할 수 있는 방법!

#### Selenium 을 사용하면 데이터 접근을 조금더 깔끔하게 할 수 있음
- 그러나, 무거움

### find_element : 단일값
### find_elements : 여러개

In [59]:
# 옵션이라는 태그를 가져옴 
gu_list = gu_list_raw.find_elements_by_tag_name('option')

# 태그를 선택했을 때, 여러개면 리스트로 나옴
gu_list

[<selenium.webdriver.remote.webelement.WebElement (session="b7c64ed185dddc214e4cfb57e5386435", element="e1a8fb2e-869e-4247-8295-798a47bc29bc")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b7c64ed185dddc214e4cfb57e5386435", element="1482b879-866c-4cf5-a40d-1fd0a8308b42")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b7c64ed185dddc214e4cfb57e5386435", element="8f1b6eb8-a174-40f1-8556-ba584b7b7295")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b7c64ed185dddc214e4cfb57e5386435", element="7aaf8c20-4b8b-4e97-81c1-d012cd5ca579")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b7c64ed185dddc214e4cfb57e5386435", element="65bd0be9-aa4d-4920-999f-3bf43a61f1d7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b7c64ed185dddc214e4cfb57e5386435", element="249087bc-fc83-4874-9870-782b12adaf4c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b7c64ed185dddc214e4cfb57e5386435", element="f099bcc2-aeb8-4c6a-a261-a3

** 컬럼을 넣을 때, 부서번호같은 일련번호로 하지 않고, 값으로 넣으면 나중에 문제가 생김!
- value가 서버에 들어가기 때문에 중요!

In [60]:
# option에서 value 추출 => [ ] (추출해서 리스트에 담는 것!)

# value가는 attribute의 값을 가져와라! 
gu_name = \
[option.get_attribute('value') for option in gu_list ]

gu_name

['',
 '강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

In [61]:
# 리스트 내의 공백 값 삭제
gu_name.remove('') # 파라미터 : 삭제할 값

In [38]:
gu_name

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

## 구별 엑셀파일 다운로드

### 강남구(gu_name[0] 엑셀파일 다운로드)

In [41]:
## for 문을 돌리기 전에 하나 연슴!  ##

# 구선택 SELECT ID : SIGUNGU_NM0
# 조회 xpath : //*[@id="searRgSelect"]/span
# 엑셀 다운로드 xpath: //*[@id="glopopd_excel"]/span

In [47]:
# 1. 구 선택
gu_select_elem = driver.find_element_by_id('SIGUNGU_NM0')

# 2. 구 값 전달 (어떤 값을 달라!) => 강남구[0]
gu_select_elem.send_keys(gu_name[0])

# () 시간 만큼 기다렸다가 시작해라 => timesleep보다 바람직!
driver.implicitly_wait(3)

# 3. 조회 버튼 선택, 클릭
xpath = '''//*[@id="searRgSelect"]/span'''
# 선택을 한 다음 -> 클릭
gu_select_search = driver.find_element_by_xpath(xpath)
gu_select_search.click() # 조회버튼 클릭


#### implicitly_wait() : 단계별로 기다렸다가 시작할 때 사용  (콜백함수)

In [46]:
# 4. 엑셀 버튼 선택, 클릭
xpath = '''//*[@id="glopopd_excel"]/span'''
gu_select_excel = driver.find_element_by_xpath(xpath)
gu_select_excel.click() # 조회버튼 클릭

# 5. 다운로드!

## 서울시 전체 구별 가격을 엑셀파일로 다운로드

In [66]:
from tqdm import tqdm
import time

for gu in tqdm(gu_name):
    driver.implicitly_wait(1)
    
    element = gu_select_elem = driver.find_element_by_id('SIGUNGU_NM0')
    element.send_keys(gu)
    
    time.sleep(1)
#     driver.implicitly_wait(5)
    
    # 3. 조회버튼 클릭
    xpath = '''//*[@id="searRgSelect"]/span'''
    # 선택을 한 다음 -> 클릭
    gu_select_search = driver.find_element_by_xpath(xpath)
    gu_select_search.click() # 조회버튼 클릭
    
    time.sleep(1)
#     driver.implicitly_wait(5)
    
    # 4. 엑셀 버튼 선택, 클릭
    xpath = '''//*[@id="glopopd_excel"]/span'''
    gu_select_excel = driver.find_element_by_xpath(xpath)
    gu_select_excel.click() # 조회버튼 클릭


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [01:37<00:00,  3.89s/it]


1. time.sleep(sec)
2. driver.implicitly.wait(sec)
3. WebDriverWait(driver, sec).until()

동적인 컨텐츠는 selenium 을 사용해야 함 (그때, try 

In [68]:
driver.close()

### 4.5 구별 가격 데이터 정리

In [67]:
import pandas as pd
from glob import glob

#### 현재 파일 25개 => 1개 읽어서 데이터 프레임으로 변환
- **25개 파일명이 들어있는 파일 목록 생성** 
- 파일 목록 생성 : **glob()**

In [89]:
glob('./data_01/지역*.xls') # 파일명이 지역으로 시작하고 .xls 인 파일

['./data_01\\지역_위치별(주유소) (1).xls',
 './data_01\\지역_위치별(주유소) (10).xls',
 './data_01\\지역_위치별(주유소) (11).xls',
 './data_01\\지역_위치별(주유소) (12).xls',
 './data_01\\지역_위치별(주유소) (13).xls',
 './data_01\\지역_위치별(주유소) (14).xls',
 './data_01\\지역_위치별(주유소) (15).xls',
 './data_01\\지역_위치별(주유소) (16).xls',
 './data_01\\지역_위치별(주유소) (17).xls',
 './data_01\\지역_위치별(주유소) (18).xls',
 './data_01\\지역_위치별(주유소) (19).xls',
 './data_01\\지역_위치별(주유소) (2).xls',
 './data_01\\지역_위치별(주유소) (20).xls',
 './data_01\\지역_위치별(주유소) (21).xls',
 './data_01\\지역_위치별(주유소) (22).xls',
 './data_01\\지역_위치별(주유소) (23).xls',
 './data_01\\지역_위치별(주유소) (24).xls',
 './data_01\\지역_위치별(주유소) (3).xls',
 './data_01\\지역_위치별(주유소) (4).xls',
 './data_01\\지역_위치별(주유소) (5).xls',
 './data_01\\지역_위치별(주유소) (6).xls',
 './data_01\\지역_위치별(주유소) (7).xls',
 './data_01\\지역_위치별(주유소) (8).xls',
 './data_01\\지역_위치별(주유소) (9).xls',
 './data_01\\지역_위치별(주유소).xls']

In [90]:
station_files = glob('./data_01/지역*.xls')
type(station_files)

list

In [91]:
# 25개 구를 반복해서 한개씩 읽어서 데이터 프레임으로 생성
# 생성된 데이터 프레임을 리스트에 저장

tmp_raw = []

for file_name in station_files:
    tmp = pd.read_excel(file_name, header=2) # 위에를 두개 지우고 가져와라
    # read_excel : 데이터 프레임이 나옴  
    tmp_raw.append(tmp) # 데이터 프레임을 리스트로 변환
    
tmp_raw

[       지역                    상호                         주소      상표  \
 0   서울특별시                구천면주유소      서울 강동구 구천면로 357 (암사동)  현대오일뱅크   
 1   서울특별시        지에스칼텍스㈜ 동서울주유소     서울 강동구 천호대로 1456 (상일동)   GS칼텍스   
 2   서울특별시         지에스칼텍스㈜ 신월주유소     서울 강동구 양재대로 1323 (성내동)   GS칼텍스   
 3   서울특별시     현대오일뱅크㈜직영 명일셀프주유소       서울 강동구 고덕로 168 (명일동)  현대오일뱅크   
 4   서울특별시     현대오일뱅크㈜직영 암사셀프주유소      서울 강동구 올림픽로 749 (암사동)  현대오일뱅크   
 5   서울특별시       (주)퍼스트오일 코알라주유소  서울특별시 강동구  올림픽로 556 (성내동)   S-OIL   
 6   서울특별시      주)지유에너지직영 오렌지주유소       서울 강동구 성안로 102 (성내동)   SK에너지   
 7   서울특별시               방아다리주유소       서울 강동구 동남로 811 (명일동)   SK에너지   
 8   서울특별시              sk해뜨는주유소      서울 강동구  상일로 132 (강일동)   SK에너지   
 9   서울특별시       대성석유(주)직영 길동주유소           서울 강동구 천호대로 1168   GS칼텍스   
 10  서울특별시        (주)삼표에너지 고덕주유소        서울 강동구 고덕로 39 (암사동)   GS칼텍스   
 11  서울특별시               천호현대주유소        서울 강동구 천중로 67 (천호동)  현대오일뱅크   
 12  서울특별시  (주)소모에너지엔테크놀러지 성내주유소      서울 강동구 올림픽로 578 (성내동)   GS칼텍스   
 13  서

In [92]:
len(tmp_raw)

25

In [93]:
station_raw = pd.concat(tmp_raw)
station_raw.head()

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,구천면주유소,서울 강동구 구천면로 357 (암사동),현대오일뱅크,02-441-0536,N,-,1583,1387,-
1,서울특별시,지에스칼텍스㈜ 동서울주유소,서울 강동구 천호대로 1456 (상일동),GS칼텍스,02-426-5372,Y,-,1595,1395,-
2,서울특별시,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),GS칼텍스,02-475-2600,N,1769,1609,1409,1000
3,서울특별시,현대오일뱅크㈜직영 명일셀프주유소,서울 강동구 고덕로 168 (명일동),현대오일뱅크,02-3428-1739,Y,1779,1615,1419,-
4,서울특별시,현대오일뱅크㈜직영 암사셀프주유소,서울 강동구 올림픽로 749 (암사동),현대오일뱅크,02-3427-2043,Y,1779,1615,1419,-


In [94]:
station_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 491 entries, 0 to 39
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      491 non-null    object
 1   상호      491 non-null    object
 2   주소      491 non-null    object
 3   상표      491 non-null    object
 4   전화번호    491 non-null    object
 5   셀프여부    491 non-null    object
 6   고급휘발유   491 non-null    object
 7   휘발유     491 non-null    object
 8   경유      491 non-null    object
 9   실내등유    491 non-null    object
dtypes: object(10)
memory usage: 42.2+ KB


In [97]:
# 상호, 주소, 셀프여부, 상표, 휘발유
stations = pd.DataFrame(
    {
          '상호':station_raw['상호']
        , '주소':station_raw['주소']
        , '가격':station_raw['휘발유']
        , '셀프여부':station_raw['셀프여부']
        , '상표':station_raw['상표']
    }
)
stations.head()

,상호,주소,가격,셀프여부,상표
0,구천면주유소,서울 강동구 구천면로 357 (암사동),1583,N,현대오일뱅크
1,지에스칼텍스㈜ 동서울주유소,서울 강동구 천호대로 1456 (상일동),1595,Y,GS칼텍스
2,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),1609,N,GS칼텍스
3,현대오일뱅크㈜직영 명일셀프주유소,서울 강동구 고덕로 168 (명일동),1615,Y,현대오일뱅크
4,현대오일뱅크㈜직영 암사셀프주유소,서울 강동구 올림픽로 749 (암사동),1615,Y,현대오일뱅크


In [98]:
# 구 컬럼 생성 : 주소에서 구 추출 => 리스트로 만듦
stations['구'] = [ addr.split()[1] for addr in stations['주소']]
stations['구']


0     강동구
1     강동구
2     강동구
3     강동구
4     강동구
     ... 
35    강남구
36    강남구
37    강남구
38    강남구
39    강남구
Name: 구, Length: 491, dtype: object

In [99]:
stations['구'].unique()

array(['강동구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
       '양천구', '영등포구', '강북구', '용산구', '은평구', '종로구', '중구', '중랑구', '강서구',
       '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '강남구'], dtype=object)

In [100]:
stations.head()

,상호,주소,가격,셀프여부,상표,구
0,구천면주유소,서울 강동구 구천면로 357 (암사동),1583,N,현대오일뱅크,강동구
1,지에스칼텍스㈜ 동서울주유소,서울 강동구 천호대로 1456 (상일동),1595,Y,GS칼텍스,강동구
2,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),1609,N,GS칼텍스,강동구
3,현대오일뱅크㈜직영 명일셀프주유소,서울 강동구 고덕로 168 (명일동),1615,Y,현대오일뱅크,강동구
4,현대오일뱅크㈜직영 암사셀프주유소,서울 강동구 올림픽로 749 (암사동),1615,Y,현대오일뱅크,강동구


In [101]:
stations = stations[stations['가격'] != '-']

In [102]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 482 entries, 0 to 36
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상호      482 non-null    object
 1   주소      482 non-null    object
 2   가격      482 non-null    object
 3   셀프여부    482 non-null    object
 4   상표      482 non-null    object
 5   구       482 non-null    object
dtypes: object(6)
memory usage: 26.4+ KB


### 가격을 숫자로 바꾸어야 함! float로 형변환!




In [106]:
stations['가격'] = [float(value) for value in stations['가격']]

In [107]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 482 entries, 0 to 36
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   상호      482 non-null    object 
 1   주소      482 non-null    object 
 2   가격      482 non-null    float64
 3   셀프여부    482 non-null    object 
 4   상표      482 non-null    object 
 5   구       482 non-null    object 
dtypes: float64(1), object(5)
memory usage: 26.4+ KB


In [108]:
stations

,상호,주소,가격,셀프여부,상표,구
0,구천면주유소,서울 강동구 구천면로 357 (암사동),1583.0,N,현대오일뱅크,강동구
1,지에스칼텍스㈜ 동서울주유소,서울 강동구 천호대로 1456 (상일동),1595.0,Y,GS칼텍스,강동구
2,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),1609.0,N,GS칼텍스,강동구
3,현대오일뱅크㈜직영 명일셀프주유소,서울 강동구 고덕로 168 (명일동),1615.0,Y,현대오일뱅크,강동구
4,현대오일뱅크㈜직영 암사셀프주유소,서울 강동구 올림픽로 749 (암사동),1615.0,Y,현대오일뱅크,강동구
...,...,...,...,...,...,...
32,갤러리아주유소,서울 강남구 압구정로 426,2015.0,N,SK에너지,강남구
33,경원주유소,서울 강남구 도산대로 428 (청담동),2016.0,N,S-OIL,강남구
34,(주)만정에너지,서울 강남구 봉은사로 433 (삼성동),2023.0,N,GS칼텍스,강남구
35,학여울주유소,서울 강남구 남부순환로 3170 (일원2동),2029.0,N,GS칼텍스,강남구


In [109]:
aaa = {'a' : [1,2,3,4]}
bbb = {'b' : [1,2,3,4]}

In [110]:
aaa_df = pd.DataFrame(aaa)
bbb_df = pd.DataFrame(bbb)

In [111]:
aaa_df

,a
0,1
1,2
2,3
3,4


In [112]:
bbb_df

,b
0,1
1,2
2,3
3,4


In [116]:
ccc_list = [aaa_df, bbb_df]
ddd_df = pd.concat(ccc_list)
ddd_df

,a,b
0,1.0,NaN
1,2.0,NaN
2,3.0,NaN
3,4.0,NaN
0,NaN,1.0
1,NaN,2.0
2,NaN,3.0
3,NaN,4.0


In [118]:
ddd_df.reset_index(inplace=True)
ddd_df

,index,a,b
0,0,1.0,NaN
1,1,2.0,NaN
2,2,3.0,NaN
3,3,4.0,NaN
4,0,NaN,1.0
5,1,NaN,2.0
6,2,NaN,3.0
7,3,NaN,4.0


In [119]:
del ddd_df['index']
ddd_df

,a,b
0,1.0,NaN
1,2.0,NaN
2,3.0,NaN
3,4.0,NaN
4,NaN,1.0
5,NaN,2.0
6,NaN,3.0
7,NaN,4.0


In [120]:
# reset인덱스가 되어 연속번호 다시 생성
stations.reset_index(inplace=True)
del stations['index']

In [121]:
stations

,상호,주소,가격,셀프여부,상표,구
0,구천면주유소,서울 강동구 구천면로 357 (암사동),1583.0,N,현대오일뱅크,강동구
1,지에스칼텍스㈜ 동서울주유소,서울 강동구 천호대로 1456 (상일동),1595.0,Y,GS칼텍스,강동구
2,지에스칼텍스㈜ 신월주유소,서울 강동구 양재대로 1323 (성내동),1609.0,N,GS칼텍스,강동구
3,현대오일뱅크㈜직영 명일셀프주유소,서울 강동구 고덕로 168 (명일동),1615.0,Y,현대오일뱅크,강동구
4,현대오일뱅크㈜직영 암사셀프주유소,서울 강동구 올림픽로 749 (암사동),1615.0,Y,현대오일뱅크,강동구
...,...,...,...,...,...,...
477,갤러리아주유소,서울 강남구 압구정로 426,2015.0,N,SK에너지,강남구
478,경원주유소,서울 강남구 도산대로 428 (청담동),2016.0,N,S-OIL,강남구
479,(주)만정에너지,서울 강남구 봉은사로 433 (삼성동),2023.0,N,GS칼텍스,강남구
480,학여울주유소,서울 강남구 남부순환로 3170 (일원2동),2029.0,N,GS칼텍스,강남구


In [123]:
import numpy as np
# 구별 평균 가격

gu_data = pd.pivot_table(
      stations
    , index=['구'] # ~별
    , values=['가격']
    , aggfunc= np.mean
)
gu_data.head()

,가격
구,
강남구,1728.270270
강동구,1657.533333
강북구,1514.307692
강서구,1573.969697
관악구,1594.750000


In [124]:
gu_data.tail()

,가격
구,
용산구,1977.428571
은평구,1535.187500
종로구,1856.333333
중구,2000.800000
중랑구,1538.687500


In [126]:
gu_data.sort_values(by='가격', ascending=False)[:5] # 내림차순

,가격
구,
중구,2000.800000
용산구,1977.428571
종로구,1856.333333
강남구,1728.270270
강동구,1657.533333


In [127]:
gu_data.sort_values(by='가격', ascending=True)[:5] # 오름차순

,가격
구,
강북구,1514.307692
성북구,1526.458333
서대문구,1535.000000
은평구,1535.187500
금천구,1537.000000
